<a href="https://colab.research.google.com/github/joony0512/Deep_Learning_Class/blob/main/Part5/P5_Ch06_CH06_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attention-based RNN Seq2Seq

In [1]:
!pip install portalocker

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import os
import sys
from datetime import datetime

drive_project_root = "/content/drive/MyDrive/#fastcampus"
sys.path.append(drive_project_root)
!pip install -r "/content/drive/MyDrive/#fastcampus/requirements.txt"

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Jul 24 07:54:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install omegaconf
!pip install torch_optimizer
!pip install wandb
!pip install efficientnet_pytorch==0.7.1
!pip install hydra-core==1.1
!pip install pytorch-lightning
!pip install --upgrade torchmetrics
!pip install --upgrade pytorch-lightning

In [5]:
# For data loading.
from typing import List
from typing import Dict
from typing import Union
from typing import Any
from typing import Optional
from typing import Iterable
from abc import abstractmethod
from abc import ABC
from datetime import datetime
from functools import partial
from collections import Counter
from collections import OrderedDict
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pprint import pprint

from torchtext import data
from torchtext import datasets
from torchtext.datasets import Multi30k
from torchtext.data.utils import get_tokenizer
from torchtext.data.functional import to_map_style_dataset
from torchtext.vocab import Vocab, build_vocab_from_iterator, vocab
import spacy

# For configuration
from omegaconf import DictConfig
from omegaconf import OmegaConf
import hydra
from hydra.core.config_store import ConfigStore

# For logger
from torch.utils.tensorboard import SummaryWriter
import wandb
os.environ["WANDB_START_METHOD"]="thread"

In [6]:

%cd /content/drive/MyDrive/#fastcampus
from data_utils import dataset_split
from config_utils import flatten_dict
from config_utils import register_config
from config_utils import configure_optimizers_from_cfg
from config_utils import get_loggers
from config_utils import get_callbacks
%cd /content


/content/drive/MyDrive/#fastcampus
/content


In [ ]:
# download eng/d data.
!python -m spacy download en
!python -m spacy download en_core_web_sm
!python -m spacy download de
!python -m spacy download de_core_news_sm

In [8]:
# practice data first go to dataconfig

# data configs
data_spacy_de_en_cfg = {
    "name": "spacy_de_en",
    "data_root": os.path.join(os.getcwd(), "data"),
    "tokenizer": "spacy",
    "src_lang": "de",
    "tgt_lang": "en",
    "src_index": 0,
    "tgt_index": 1,
    "vocab": {
        "special_symbol2index": {
            # Define special symbols and indices
            # UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
            # Make sure the tokens are in order of their indices to properly insert them in vocab
            '<unk>': 0,
            '<pad>': 1,
            '<bos>': 2,
            '<eos>': 3,
        },
        "special_first": True,
        "min_freq": 2
    }
}

data_cfg = OmegaConf.create(data_spacy_de_en_cfg)



In [9]:
# get dataset
# data_root = os.path.join(os.getcwd(), "data")

train_data, valid_data, test_data = Multi30k(data_cfg.data_root)

test_data = to_map_style_dataset(test_data)


In [10]:
# Create source and target language tokenizer. Make sure to install the dependencies.
# pip install -U spacy
# python -m spacy download en_core_web_sm
# python -m spacy download de_core_news_sm

def get_token_transform(data_cfg: DictConfig) -> dict:
    token_transform = {}
    token_transform[data_cfg.src_lang] = get_tokenizer(
        data_cfg.tokenizer, language=data_cfg.src_lang
    )
    token_transform[data_cfg.tgt_lang] = get_tokenizer(
        data_cfg.tokenizer, language=data_cfg.tgt_lang
    )
    return token_transform

token_transform = get_token_transform(data_cfg)

/usr/local/lib/python3.10/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "de" could not be loaded, trying "de_core_news_sm" instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


In [11]:
# helper function to yield list of tokens
def yield_tokens(
    data_iter: Iterable, lang: str, lang2index: Dict[str, int]
) -> List[str]:

    for data_sample in data_iter:
        yield token_transform[lang](data_sample[lang2index[lang]])

def get_vocab_transform(data_cfg: DictConfig) -> dict:
    vocab_transform = {}
    for ln in [data_cfg.src_lang, data_cfg.tgt_lang]:
        # Training data Iterator
        train_iter = Multi30k(
            split='train', language_pair=(data_cfg.src_lang, data_cfg.tgt_lang)
        )
        # Create torchtext's Vocab object
        vocab_transform[ln] = build_vocab_from_iterator(
            yield_tokens(
                train_iter,
                ln,
                {
                    data_cfg.src_lang: data_cfg.src_index,
                    data_cfg.tgt_lang: data_cfg.tgt_index
                }
            ),
            min_freq=data_cfg.vocab.min_freq,
            specials=list(data_cfg.vocab.special_symbol2index.keys()),
            special_first=data_cfg.vocab.special_first,
        )

    # Set UNK_IDX as the default index. This index is returned when the token is not found.
    # If not set, it throws RuntimeError when the queried token is not found in the Vocabulary.
    for ln in [data_cfg.src_lang, data_cfg.tgt_lang]:
        vocab_transform[ln].set_default_index(
            data_cfg.vocab.special_symbol2index["<unk>"]
        )
    return vocab_transform

vocab_transform = get_vocab_transform(data_cfg)

In [12]:
print(vocab_transform["de"]["<unk>"])
print(vocab_transform["en"]["<unk>"])
print(vocab_transform["en"]["hello"], vocab_transform["en"]["world"])

0
0
5465 1870


In [13]:
# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int], bos_index: int, eos_index: int):
    return torch.cat((torch.tensor([bos_index]),
                      torch.tensor(token_ids),
                      torch.tensor([eos_index])))

# src and tgt language text transforms to convert raw strings into tensors indices
def get_text_transform(data_cfg):
    text_transform = {}
    for ln in [data_cfg.src_lang, data_cfg.tgt_lang]:
        text_transform[ln] = sequential_transforms(
            token_transform[ln], #Tokenization
            vocab_transform[ln], #Numericalization
            partial(
                tensor_transform,
                bos_index=data_cfg.vocab.special_symbol2index["<bos>"],
                eos_index=data_cfg.vocab.special_symbol2index["<eos>"],
            )
        ) # Add BOS/EOS and create tensor
    return text_transform

text_transform = get_text_transform(data_cfg)

In [14]:
print(text_transform["en"]("hello"))
print(text_transform["en"]("hello,"))
print(text_transform["en"]("hello, how"))
print(text_transform["en"]("hello, how are you ?"))

tensor([   2, 5465,    3])
tensor([   2, 5465,   15,    3])
tensor([   2, 5465,   15,  889,    3])
tensor([   2, 5465,   15,  889,   17, 1328, 2470,    3])


In [15]:
# function to collate data samples into batch tesors
def collate_fn(batch, data_cfg: DictConfig):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[data_cfg.src_lang](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[data_cfg.tgt_lang](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=data_cfg.vocab.special_symbol2index["<pad>"])
    tgt_batch = pad_sequence(tgt_batch, padding_value=data_cfg.vocab.special_symbol2index["<pad>"])
    return src_batch, tgt_batch

def get_collate_fn(cfg: DictConfig):
    return partial(collate_fn, data_cfg=cfg.data)

def get_multi30k_dataloader(
    split_mode: str, language_pair, batch_size: int, collate_fn
):
    iter = Multi30k(split=split_mode, language_pair=language_pair)
    dataset = to_map_style_dataset(iter)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, collate_fn=collate_fn
    )
    return dataloader


In [16]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [17]:
def _text_postprocessing(res: List[str]) -> str:
    if "<eos>" in res:
        res = res[:res.index("<eos>")]
    if "<pad>" in res:
        res = res[:res.index("<pad>")]
    res = " ".join(res).replace("<bos>", "")
    return res

class BaseTranslateLightningModule(pl.LightningModule):
    def __init__(self, cfg: DictConfig):
        super().__init__()
        self.cfg = cfg
        self.loss_function = torch.nn.CrossEntropyLoss(
            ignore_index=cfg.data.vocab.special_symbol2index["<pad>"]
        )

    def configure_optimizers(self):
        self._optimizers, self._schedulers = configure_optimizers_from_cfg(
            self.cfg, self
        )
        return self._optimizers, self._schedulers

    @abstractmethod
    def forward(x):
        raise NotImplementedError()

    def _forward(self, src, tgt, mode: str, teacher_forcing_ratio: float = 0.5):

        assert mode in ["train", "val", "test"]

        # get predictions
        tgt_inputs = tgt[:-1, :] # delete ends...
        outputs = self(src, tgt_inputs, teacher_forcing_ratio=teacher_forcing_ratio)
        tgt_outputs = tgt[1:, :]  # remove start tokens..

        loss = self.loss_function(
            outputs.reshape(-1, outputs.shape[-1]),
            tgt_outputs.reshape(-1)
        )

        logs_detail = {
            f"{mode}_src": src,
            f"{mode}_tgt": tgt,
            f"{mode}_results": outputs,
        }

        if mode in ["val", "test"]:
            _, tgt_results = torch.max(outputs, dim=2)

            src_texts = []
            tgt_texts = []
            res_texts = []

            for src_i in torch.transpose(src, 0, 1).detach().cpu().numpy().tolist():
                res = vocab_transform[self.cfg.data.src_lang].lookup_tokens(src_i)
                src_texts.append(_text_postprocessing(res))

            for tgt_i in torch.transpose(tgt, 0, 1).detach().cpu().numpy().tolist():
                res = vocab_transform[self.cfg.data.tgt_lang].lookup_tokens(tgt_i)
                tgt_texts.append(_text_postprocessing(res))

            for tgt_res_i in torch.transpose(tgt_results, 0, 1).detach().cpu().numpy().tolist():
                res = vocab_transform[cfg.data.tgt_lang].lookup_tokens(tgt_res_i)
                res_texts.append(_text_postprocessing(res))

            text_result_summary = {
                f"{mode}_src_text": src_texts,
                f"{mode}_tgt_text": tgt_texts,
                f"{mode}_results_text": res_texts,
            }
            print(f"{self.global_step} step: \n src_text: {src_texts[0]}, \n tgt_text: {tgt_texts[0]}, \n result_text:{res_texts[0]}")
            logs_detail.update(text_result_summary)

        return {f"{mode}_loss": loss}, logs_detail

    def training_step(self, batch, batch_idx):
        src, tgt = batch[0], batch[1]

        logs, logs_detail = self._forward(src, tgt, "train", self.cfg.model.teacher_forcing_ratio)
        self.log_dict(logs)
        logs["loss"] = logs["train_loss"]
        return logs

    def validation_step(self, batch, batch_idx):
        src, tgt = batch[0], batch[1]
        logs, logs_detail = self._forward(src, tgt, "val", 0.0)
        self.log_dict(logs)
        logs["loss"] = logs["val_loss"]
        logs.update(logs_detail)

        return logs

    def test_step(self, batch, batch_idx):
        src, tgt = batch[0], batch[1]
        logs, logs_detail = self._forward(images, labels, "test", 0.0)
        self.log_dict(logs)
        logs["loss"] = logs["test_loss"]
        logs.update(logs_detail)
        # wandb_logger, tensorboard_logger = self.logger.experiment
        # wandb_logger.log(logs_detail)
        # self.log_dict(logs)
        return logs


In [18]:
# weight initialization
def init_weights(model: Union[nn.Module, pl.LightningModule]):
    for name, param in model.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


In [19]:
class LSTMEncoder(nn.Module):
    def __init__(
        self,
        input_dim: int,
        embed_dim: int,
        hidden_dim: int,
        n_layers: int,
        dropout: float
    ):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, embed_dim)
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout = dropout)

        self.dropout = nn.Dropout(dropout)

        self.apply(init_weights)

    def forward(self, src):
        # src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))

        # embedded = [src len, batch size, emb dim]
        outputs, (hidden, cell) = self.rnn(embedded)

        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]

        # outputs are always from the top hidden layer
        return hidden, cell


class LSTMDecoder(nn.Module):
    def __init__(
        self,
        output_dim: int,
        embed_dim: int,
        hidden_dim: int,
        n_layers: int,
        dropout: float,
    ):
        super().__init__()

        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, embed_dim)
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]

        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hid dim]
        # context = [n layers, batch size, hid dim]

        input = input.unsqueeze(0)

        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))

        # embedded = [1, batch size, emb dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        # output = [seq len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]

        # seq len and n directions will always be 1 in the decoder, therefore:
        # output = [1, batch size, hid dim]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]

        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]

        return prediction, hidden, cell


class LSTMSeq2Seq(BaseTranslateLightningModule):
    def __init__(self, cfg: DictConfig):
        super().__init__(cfg)
        # encoder, decoder, device

        self.encoder = LSTMEncoder(**cfg.model.enc)
        self.decoder = LSTMDecoder(**cfg.model.dec)
        # self.device = device

        assert self.encoder.hidden_dim == self.decoder.hidden_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert self.encoder.n_layers == self.decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"

        self.apply(init_weights)

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        # src = [src len, batch size]
        # trg = [trg len, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)

        #first input to the decoder is the <sos> tokens
        input = trg[0,:]

        for t in range(1, trg_len):

            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            #get the highest predicted token from our predictions
            top1 = output.argmax(1)

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs


## GRU + Bidirectional

In [31]:

# Concat(Addictive) Attention 기반의 모델 새로 정의
# encoder decoder RNN 이 다를 수 있다
class BidirectionalGRUEncoder(nn.Module):
    def __init__(
            self,
            input_dim: int,
            embed_dim: int,
            enc_hidden_dim: int,
            dec_hidden_dim: int,
            n_layers: int,
            dropout: float,
    ):
        super().__init__()
        self.input_dim = input_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, embed_dim)
        self.rnn = nn.GRU(embed_dim, enc_hidden_dim, n_layers, bidirectional = True, dropout = dropout)
        self.fc = nn.Linear(enc_hidden_dim * 2 , dec_hidden_dim)
        self.dropout = nn.Dropout(dropout)

        # initialization of weights
        self.apply(init_weights)

    def forward(self, src):
        # src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))

        # embedded = [src len, batch size, emb dim]
        outputs, hidden = self.rnn(embedded) # lstm과 달리 hidden으로 받는다

        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]

        # hidden -> [forward_1, barchward_1, forward_2, backward_2..]
        # 우리가 필요한것은 맨위 마지막 layer의 forward backward 두개 concat 한개 필요
        # =>torch.cat((hidden[-2, : , :], hidden[-1, : , :]), dim =1)

        # encoder RNNs fed through a linear layer to connect decoder.
        hidden = torch.tanh(self.fc(
            torch.cat((hidden[-2, : , :], hidden[-1, : , :]), dim =1)
        ))
        return outputs, hidden


class ConcatAttention(nn.Module):
    def __init__(self, enc_hidden_dim : int, dec_hidden_dim : int):
        super().__init__()

        self.attn = nn.Linear((enc_hidden_dim * 2) + dec_hidden_dim, dec_hidden_dim)
        self.v = nn.Linear(dec_hidden_dim, 1, bias = False)

    def forward(self, hidden, encoder_outputs):
        # hidden = [batch_size, dec_hidden_dim] => from decoder (query)
        # encoder_outputs = [src_len, batch_len, enc_hidden_dim * 2] (key, value)

        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]

        # repeat decoder hidden state src_len times ...
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        # hidden : [batch_size, src_len, dec_hidden_dim]
        # encoder_outputs = [batch_dize, src_len, enc_hidden_dim *2]

        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))

        # energy = [batch_size, src_len, dec_hidden_dim]

        attention = self.v(energy).squeeze(2)

        # attention = [batch_size, src_len]

        return F.softmax(attention, dim =1)

class AttentionRNNDecoder(nn.Module):
    def __init__(
        self,
        output_dim: int,
        embed_dim: int,
        enc_hidden_dim: int,
        dec_hidden_dim: int,
        n_layers: int,
        dropout: float,
        attention : nn.Module
    ):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, embed_dim)
        self.rnn = nn.GRU((enc_hidden_dim * 2) + embed_dim, dec_hidden_dim, n_layers, dropout = dropout)
        self.fc_out = nn.Linear((enc_hidden_dim *2) + dec_hidden_dim + embed_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):

        # input [batch size] # start token
        # hidden [batch_size, dec_hidden_dim]
        # encoder_outputs [src_len, batch_size, enc_hidden_dim*2]

        input = input.unsqueeze(0) # input = [1, batch_size]

        embedded = self.dropout(self.embedding(input)) # [1, batch_size, embed_dim]

        a = self.attention(hidden, encoder_outputs) # [batch_size, src_len]
        a = a.unsqueeze(1) # [batch_size, 1, src_len]

        encoder_outputs = encoder_outputs.permute(1,0,2) #[batch_size, src_len, enc_hidden_dim *2]
        weighted = torch.bmm(a, encoder_outputs) # [batch_size, 1, enc_hidden_dim*2]

        weighted = weighted.permute(1,0,2) # [1, batch_size, enc_hidden_dim *2]

        rnn_input = torch.cat((embedded, weighted), dim = 2) # [1, batch_dize, (enc_hidden_dim *2 + embed_dim)]

        # hidden.unsqueeze : [1, batch_size, dec_hidden_dim]
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        # output = [seq_len, batch_size , dec_hidden_dim * n directions] => [1, batch_size, dec_hidden_dim]
        # hidden = [n layers * n_directions, batch_size, dec_hidden_dim] => [1, batch_size, dec_hidden_dim]

        if not (output == hidden).all():
            raise AssertionError()

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)

        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1)) # [batch_size, output_dim]

        return prediction, hidden.squeeze(0)

class AttentionBasedSeq2Seq(BaseTranslateLightningModule):
    def __init__(self, cfg : DictConfig):
        super().__init__(cfg)
        self.encoder = BidirectionalGRUEncoder(**cfg.model.enc)
        self.attention = ConcatAttention(**cfg.model.attention)
        self.decoder = AttentionRNNDecoder(
            attention = self.attention, **cfg.model.dec
        )


    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

            # src = [src len, batch size]
            # trg = [trg len, batch size]
            # teacher_forcing_ratio is probability to use teacher forcing
            # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

            batch_size = trg.shape[1]
            trg_len = trg.shape[0]
            trg_vocab_size = self.decoder.output_dim

            #tensor to store decoder outputs
            outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

            #last hidden state of the encoder is used as the initial hidden state of the decoder
            encoder_outputs, hidden = self.encoder(src)

            #first input to the decoder is the <sos> tokens
            input = trg[0,:]

            for t in range(1, trg_len):

                #insert input token embedding, previous hidden and previous cell states
                #receive output tensor (predictions) and new hidden and cell states
                output, hidden = self.decoder(input, hidden, encoder_outputs)

                #place predictions in a tensor holding predictions for each token
                outputs[t] = output

                #decide if we are going to use teacher forcing or not
                teacher_force = random.random() < teacher_forcing_ratio

                #get the highest predicted token from our predictions
                top1 = output.argmax(1)

                #if teacher forcing, use actual next token as next input
                #if not, use predicted token
                input = trg[t] if teacher_force else top1

            return outputs



In [32]:

# data configs
data_spacy_de_en_cfg = {
    "name": "spacy_de_en",
    "data_root": os.path.join(os.getcwd(), "data"),
    "tokenizer": "spacy",
    "src_lang": "de",
    "tgt_lang": "en",
    "src_index": 0,
    "tgt_index": 1,
    "vocab": {
        "special_symbol2index": {
            # Define special symbols and indices
            # UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
            # Make sure the tokens are in order of their indices to properly insert them in vocab
            '<unk>': 0,
            '<pad>': 1,
            '<bos>': 2,
            '<eos>': 3,
        },
        "special_first": True,
        "min_freq": 2
    }
}

data_cfg = OmegaConf.create(data_spacy_de_en_cfg)

# get dataset
# data_root = os.path.join(os.getcwd(), "data")
train_data, valid_data, test_data = Multi30k(data_cfg.data_root)

token_transform = get_token_transform(data_cfg)
vocab_transform = get_vocab_transform(data_cfg)

/usr/local/lib/python3.10/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "de" could not be loaded, trying "de_core_news_sm" instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


In [33]:
# model configs
model_translate_lstm_seq2seq_cfg = {
    "name": "LSTMSeq2Seq",
    "out_dim": len(vocab_transform[data_cfg.src_lang]),
    "enc": {
        "input_dim": len(vocab_transform[data_cfg.src_lang]),
        "embed_dim": 256,
        "hidden_dim": 256,
        "n_layers": 2,
        "dropout": 0.5,
    },
    "dec": {
        "output_dim": len(vocab_transform[data_cfg.tgt_lang]),
        "embed_dim": 256,
        "hidden_dim": 256,
        "n_layers": 2,
        "dropout": 0.5,
    },
    "teacher_forcing_ratio": 0.5
}

model_translate_attention_based_seq2seq_cfg = {
    "name": "AttentionBasedSeq2Seq",
    "enc": {
        "input_dim": len(vocab_transform[data_cfg.src_lang]),
        "embed_dim": 256,
        "enc_hidden_dim": 512,
        "dec_hidden_dim": 512,
        "n_layers": 1,
        "dropout": 0.5,
    },
    "dec": {
        "output_dim": len(vocab_transform[data_cfg.tgt_lang]),
        "embed_dim": 256,
        "enc_hidden_dim": 512,
        "dec_hidden_dim": 512,
        "n_layers": 1,
        "dropout": 0.5,
    },
    "attention" : {
        "enc_hidden_dim": 512,
        "dec_hidden_dim": 512,
    },
    "teacher_forcing_ratio": 0.5
}


# optimizer configs
opt_cfg = {
    "optimizers": [
        {
            "name": "RAdam",
            "kwargs": {
                "lr": 1e-3,
            }
        }
    ],
    "lr_schedulers": [
        {
            "name": None,
            "kwargs": {
                "warmup_end_steps": 1000
            }
        },
    ]
}

_merged_cfg_presets = {
    "LSTM_seq2seq_de_en_translate": {
        "opt": opt_cfg,
        "data": data_spacy_de_en_cfg,
        "model": model_translate_lstm_seq2seq_cfg,
    },
    "attention_based_seq2seq_de_en_translate": {
        "opt": opt_cfg,
        "data": data_spacy_de_en_cfg,
        "model": model_translate_attention_based_seq2seq_cfg,
    },
}

# clear config instance first
hydra.core.global_hydra.GlobalHydra.instance().clear()

# register preset configs
register_config(_merged_cfg_presets)

# initialize & make config
## select mode here ##
# .................. #
hydra.initialize(config_path=None)
cfg = hydra.compose("attention_based_seq2seq_de_en_translate")

# override some cfg
run_name = f"{datetime.now().isoformat(timespec='seconds')}-{cfg.model.name}-{cfg.data.name}"

# Define other train configs & log_configs
# Merge configs into one & register it to Hydra.
project_root_dir = os.path.join(
    drive_project_root, "runs", "de_en_translate"
)
save_dir = os.path.join(project_root_dir, run_name)
run_root_dir = os.path.join(project_root_dir, run_name)
# train configs
train_cfg ={
    'train_batch_size' : 128,
    'val_batch_size' : 32,
    'test_batch_size' : 32,
    'train_val_split' : [0.9,0.1],
    'run_root_dir' : run_root_dir,
    'trainer_kwargs' : {
        'accelerator': 'gpu',
        'num_nodes' : 0,
        'max_epochs' :50,
        'val_check_interval': 1.0, #train 1epoch당 val 1회
        'log_every_n_steps' : 100,
        # 'flush_logs_every_n_steps' : 100, #100번 step마다
    }

}
# logger configs
log_cfg = {
    'loggers' : {
        'WandbLogger' : {
            'project' : 'fastcampus_de_en_translate_tutorials',
            'name' : run_name,
            'tags' : ['fastcampus_de_en_translate_tutorials'],
            'save_dir' : run_root_dir,

        },
        'TensorBoardLogger' : {
            'save_dir' : project_root_dir,
            'name' : run_name,
        }
    },
    'callbacks' : {
        'ModelCheckpoint' : {
            'save_top_k' : 3,
            'monitor' : 'val_loss',
            'mode' : 'min',
            'verbose' : True,
            'dirpath' : os.path.join(run_root_dir, 'weights'),
            'filename' : '{epoch}-{val_loss:.3f}',

        },
        'EarlyStopping' : {
            'monitor' : 'val_loss',
            'mode' : 'min',
            'patience' : 3,
            'verbose' : True
        }
    }
}

# unlock config & set train, log config
OmegaConf.set_struct(cfg, False)
cfg.train =train_cfg
cfg.log = log_cfg

# lock config
OmegaConf.set_struct(cfg, True)
print(OmegaConf.to_yaml(cfg))



opt:
  optimizers:
  - name: RAdam
    kwargs:
      lr: 0.001
  lr_schedulers:
  - name: null
    kwargs:
      warmup_end_steps: 1000
data:
  name: spacy_de_en
  data_root: /content/data
  tokenizer: spacy
  src_lang: de
  tgt_lang: en
  src_index: 0
  tgt_index: 1
  vocab:
    special_symbol2index:
      <unk>: 0
      <pad>: 1
      <bos>: 2
      <eos>: 3
    special_first: true
    min_freq: 2
model:
  name: AttentionBasedSeq2Seq
  enc:
    input_dim: 8014
    embed_dim: 256
    enc_hidden_dim: 512
    dec_hidden_dim: 512
    n_layers: 1
    dropout: 0.5
  dec:
    output_dim: 6191
    embed_dim: 256
    enc_hidden_dim: 512
    dec_hidden_dim: 512
    n_layers: 1
    dropout: 0.5
  attention:
    enc_hidden_dim: 512
    dec_hidden_dim: 512
  teacher_forcing_ratio: 0.5
train:
  train_batch_size: 128
  val_batch_size: 32
  test_batch_size: 32
  train_val_split:
  - 0.9
  - 0.1
  run_root_dir: /content/drive/MyDrive/#fastcampus/runs/de_en_translate/2023-07-24T08:08:02-AttentionBased

In [34]:
# dataloader def
train_dataloader = get_multi30k_dataloader(
    "train",
    (cfg.data.src_lang, cfg.data.tgt_lang),
    cfg.train.train_batch_size,
    collate_fn=get_collate_fn(cfg)
)
val_dataloader = get_multi30k_dataloader(
    "valid",
    (cfg.data.src_lang, cfg.data.tgt_lang),
    cfg.train.val_batch_size,
    collate_fn=get_collate_fn(cfg)
)
test_dataloader = get_multi30k_dataloader(
    "test",
    (cfg.data.src_lang, cfg.data.tgt_lang),
    cfg.train.test_batch_size,
    collate_fn=get_collate_fn(cfg)
)

In [35]:
# model definition
def get_pl_model(cfg: DictConfig, checkpoint_path: Optional[str] = None):
    if cfg.model.name == "LSTMSeq2Seq":
        model = LSTMSeq2Seq(cfg)
    elif cfg.model.name =="AttentionBasedSeq2Seq":
        model =AttentionBasedSeq2Seq(cfg)
    else:
        raise NotImplementedError("not implemented model")

    if checkpoint_path is not None:
        model = model.load_from_checkpoint(checkpoint_path=checkpoint_path)
    return model

model = None
model = get_pl_model(cfg)
print(model)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


AttentionBasedSeq2Seq(
  (loss_function): CrossEntropyLoss()
  (encoder): BidirectionalGRUEncoder(
    (embedding): Embedding(8014, 256)
    (rnn): GRU(256, 512, dropout=0.5, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (attention): ConcatAttention(
    (attn): Linear(in_features=1536, out_features=512, bias=True)
    (v): Linear(in_features=512, out_features=1, bias=False)
  )
  (decoder): AttentionRNNDecoder(
    (attention): ConcatAttention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(6191, 256)
    (rnn): GRU(1280, 512, dropout=0.5)
    (fc_out): Linear(in_features=1792, out_features=6191, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)


In [36]:
# pytorch-lightning trainer def

logger = get_loggers(cfg)
callbacks = get_callbacks(cfg)

trainer =pl.Trainer(
    callbacks = callbacks,
    logger = logger,
    default_root_dir= cfg.train.run_root_dir,
    num_sanity_val_steps=2,
    **cfg.train.trainer_kwargs
)



wandb: Currently logged in as: hyejun12123. Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [ ]:
trainer.fit(model, train_dataloader, val_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type                    | Params
----------------------------------------------------------
0 | loss_function | CrossEntropyLoss        | 0     
1 | encoder       | BidirectionalGRUEncoder | 4.9 M 
2 | attention     | ConcatAttention         | 787 K 
3 | decoder       | AttentionRNNDecoder     | 16.2 M
----------------------------------------------------------
21.2 M    Trainable params
0         Non-trainable params
21.2 M    Total params
84.681    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

0 step: 
 src_text:  Eine Gruppe von Männern lädt <unk> auf einen Lastwagen, 
 tgt_text:  A group of men are loading cotton onto a truck, 
 result_text:<unk> television downwards toil Capris casting wrapping Cricket fake buoy loader hurt waking Factory skier process remember Lone Welcome briefs Rocks tents Vendors butcher milk mounds
0 step: 
 src_text:  Kind spielt auf einem Spielplatz und hängt dabei an Stangen ., 
 tgt_text:  Child playing on a playground , hanging from bars ., 
 result_text:<unk> television downwards toil Capris casting wrapping Cricket fake buoy loader hurt waking Factory skier process remember Lone Welcome briefs Rocks tents Vendors butcher milk mounds screwdriver almost


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 30. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 35. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

...

227 step: 
 src_text:  Ein kleiner Junge zeigt seine <unk> <unk> ., 
 tgt_text:  A young boy shows his brown and green bead necklace ., 
 result_text:<unk> man in a a a a .
227 step: 
 src_text:  Der Junge springt mit einem <unk> aus dem Bett ., 
 tgt_text:  The boy leaps of his bed with a karate kick ., 
 result_text:<unk> man in a a a a a . .
227 step: 
 src_text:  Zwei Männer aus gegnerischen Teams rennen in Richtung eines Fußballs ., 
 tgt_text:  Two men on opposing teams race toward a soccer ball ., 
 result_text:<unk> man in a a a a a a .
227 step: 
 src_text:  Zwei schwarz gekleidete Männer mit einer grünen und einer roten Fliege treten vor einer Menschenmenge auf ., 
 tgt_text:  Two men in black clothes with blue and red bowties are performing in front of a crowd ., 
 result_text:<unk> man in a a a a a a a a a a .


Validation: 0it [00:00, ?it/s]

454 step: 
 src_text:  Eine Gruppe von Männern lädt <unk> auf einen Lastwagen, 
 tgt_text:  A group of men are loading cotton onto a truck, 
 result_text:<unk> group of are a a a . .
454 step: 
 src_text:  Kind spielt auf einem Spielplatz und hängt dabei an Stangen ., 
 tgt_text:  Child playing on a playground , hanging from bars ., 
 result_text:<unk> man is a a a a a . .
454 step: 
 src_text:  Eine bunt gekleidete Frau geht an einem weißen Lastwagen vorbei , der mit Flaschen gefüllt ist ., 
 tgt_text:  A woman in a colorful outfit is walking by a white truck filled with bottles ., 
 result_text:<unk> man in a a shirt is a a a a a . .
454 step: 
 src_text:  Zwei Kinder graben Löcher in die Erde ., 
 tgt_text:  <unk> children dig holes in the dirt ., 
 result_text:<unk> group of are a a a . .
454 step: 
 src_text:  Mann in einem kleinen weißen Boot auf einem See ., 
 tgt_text:  Man in a small white boat on a lake ., 
 result_text:<unk> man in a a shirt is a a . .
454 step: 
 src_text: 

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved by 0.141 >= min_delta = 0.0. New best score: 5.229
INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 454: 'val_loss' reached 5.22931 (best 5.22931), saving model to '/content/drive/MyDrive/#fastcampus/runs/de_en_translate/2023-07-24T08:08:02-AttentionBasedSeq2Seq-spacy_de_en/weights/epoch=1-val_loss=5.229.ckpt' as top 3


454 step: 
 src_text:  Zwei Männer aus gegnerischen Teams rennen in Richtung eines Fußballs ., 
 tgt_text:  Two men on opposing teams race toward a soccer ball ., 
 result_text:<unk> group of are a a a a a . .
454 step: 
 src_text:  Zwei schwarz gekleidete Männer mit einer grünen und einer roten Fliege treten vor einer Menschenmenge auf ., 
 tgt_text:  Two men in black clothes with blue and red bowties are performing in front of a crowd ., 
 result_text:<unk> man in a a shirt and a a a a a a a a a . .
